# Mix the tunes

In [1]:
import numpy as np 
import pandas as pd
import ffmpeg
from pathlib import Path

import glob
import os
import shutil

import IPython.display as ipd
from scipy.io import wavfile
from scipy.signal import resample

In [2]:
import librosa

import torchaudio
import torch
import torchaudio.functional as F
from torchaudio.transforms import PitchShift

### Pull all the available wavs that you can work with

In [3]:
def pull_available_wavs(source_dir, target_dir, extension):
    source_dir = os.path.expanduser(source_dir)
    target_dir = os.path.expanduser(target_dir)
    
    # Create target directory if it doesn't exist, make pattern to find wavs via glob match
    os.makedirs(target_dir, exist_ok=True)
    pattern = os.path.join(source_dir, f"*.{extension}")
    matching_files = glob.glob(pattern, recursive=False)

    # Add each file to a list
    available_wavs = [f for f in glob.glob(pattern) if os.path.isfile(f)]
    
    # Copy each file to target directory
    for file_path in matching_files:
        if os.path.isfile(file_path):
            filename = os.path.basename(file_path)
            target_path = os.path.join(target_dir, filename)
            shutil.copy2(file_path, target_path)

    return available_wavs

avail_wavs = pull_available_wavs("~/ohw25_proj_RiptideRemix/data/template_sounds", "~/ohw25_proj_RiptideRemix/contributor_folders/isabelle/audio_processing_mixers/available_wavs", "wav")

In [4]:
print("Playing ", avail_wavs[5])
ipd.Audio(avail_wavs[5])

Playing  /home/jovyan/ohw25_proj_RiptideRemix/data/template_sounds/bowhead_template.wav


### Loop file as many times as user requests, but will cut off at 60 seconds

In [5]:
def loop(input_file, output_file, num_loops, max_duration=60):
    sr, y = wavfile.read(input_file)

    num_samples = y.shape[0]
    max_samples = int(max_duration * sr)

    if y.ndim == 1:
        y_looped = np.tile(y, num_loops)
    else:
        y_looped = np.tile(y, (num_loops, 1))

    y_final = y_looped[:max_samples]

    wavfile.write(output_file, sr, y_final.astype(y.dtype))

    return 

loop(avail_wavs[5], "loop_exp.wav", 3)

In [6]:
ipd.Audio("loop_exp.wav")

### Increase speed / pitch

In [7]:
def pitch(input_file, output_file, n_steps):

    y, sr = librosa.load(input_file, sr=None, mono=False)
    y_shifted = librosa.effects.pitch_shift(y=y, sr=sr, n_steps=n_steps)

    wavfile.write(output_file, sr, y_shifted)
    return
    
pitch(avail_wavs[5], "pitch_exp.wav", 10)

In [8]:
def speed(input_file, output_file, factor):
    audio = AudioSegment.from_wav(input_file)
    slower_audio = audio._spawn(audio.raw_data, overrides={"frame_rate": int(audio.frame_rate * factor)})
    slower_audio = slower_audio.set_frame_rate(audio.frame_rate)
    slower_audio.export(output_file, format="wav")
    
    return slower_audio

pitch(avail_wavs[5], "pitch_exp.wav", 1)

In [55]:
ipd.Audio("pitch_exp.wav")

### Increase amplitude

In [9]:
def amplitude(input_file, output_file, gain_factor):
    sr, y = wavfile.read(input_file)
    print(y[:10])
    
    y_amplified = y + gain_factor
    #print(y_amplified[:10])
    
    wavfile.write(output_file, sr, y)
    return

amplitude(avail_wavs[5], "amp_exp.wav", 1)

[-80  48  64  96 112  80  64 144  80  64]


In [10]:
ipd.Audio("amp_exp.wav")

In [4]:
#fix heightened background noise